In [1]:
%matplotlib inline
%matplotlib widget
#%matplotlib notebook
import json
import os
import pprint
import datetime as dt
from matplotlib import pyplot as plt
from matplotlib.container import BarContainer

import numpy as np
import math
import pandas as pd
import textwrap

import ipywidgets as widgets
from ipywidgets import Label, HBox, VBox, Button, HTML
from ipywidgets import interact, interact_manual, Layout
import IPython.display
from IPython.display import display, clear_output
import mplcursors
from matplotlib.legend_handler import HandlerLine2D
from collections import Counter
from matplotlib.animation import FuncAnimation
import matplotlib.dates as mdates 
import copy
from matplotlib.dates import DateFormatter
plt.style.use('seaborn-notebook')


from query import * 
from dataframe_visualization import *
from performance_analysis import *
from visualize_widgets import visualize


In [2]:
# dataframe을 얻기 위함
import matplotlib

df = sns.load_dataset('iris')

#grid = sns.FacetGrid(df, col = 'species', margin_titles = True)
#grid.map(sns.lmplot, 'species', 'sepal_length', fit_reg = False)

#sns.displot(df, x ='sepal_width',y = 'sepal_length', kind ='line')
# df = q("""SELECT
#     EVENT_NAME,
#     MAX_TIMER_READ,
#     AVG_TIMER_READ,
#     MAX_TIMER_WRITE,
#     AVG_TIMER_WRITE,
#     MAX_TIMER_MISC,
#     AVG_TIMER_MISC
#     FROM performance_schema.file_summary_by_event_name;
# """)

In [9]:
import_and_update_data() # to import data from collected files

In [2]:
visualize() # show widgets for visualization